In [4]:
! echo $LD_LIBRARY_PATH

In [5]:
! export LD_LIBRARY_PATH=/usr/lib/wsl/lib:$LD_LIBRARY_PATH

In [2]:
# os.environ["LD_LIBRARY_PATH"] = '/usr/lib/wsl/lib:'

In [3]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = '1'

import torch

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
torch.cuda.current_device()

0

In [4]:
# device = 'cpu'

In [5]:
torch.cuda.get_device_name(0)

'NVIDIA RTX A5000'

In [6]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision

from tqdm.notebook import tqdm
import numpy as np
import json
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
from os.path import join, exists
import copy
import matplotlib.pyplot as plt

import sys
# sys.path.append('../dataset/safegraph/utils/')
sys.path.append('utils/')
from dataset_classes import * 
from models import *
from helper_funcs import *

In [7]:
def plot_curves(loss, metric, best_epoch_loss, best_epoch_metric):
    plt.figure()
    plt.plot([epoch for epoch in range(0, len(loss))], loss, marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'Train Stage: {data_type}')
    plt.axvline(x = best_epoch_loss, color = 'r', linestyle='-')
    plt.savefig(save_dir + 'loss')

    plt.figure()
    plt.plot([epoch for epoch in range(0, len(metric))], metric, marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (of triplet learning)')
    plt.title(f'Train Stage: {data_type}')
    plt.axvline(x = best_epoch_metric, color = 'r', linestyle='-')
    plt.savefig(save_dir + 'accuracy')
    
def metrics(stats):
    """
    Self-defined metrics function to evaluate and compare models
    stats: {'TP': TP, 'FP': FP, 'TN': TN, 'FN': FN}
    return: must be a single number """
    accuracy = (stats['T'] + 0.00001) * 1.0 / (stats['T'] + stats['F'] + 0.00001)
    # print(accuracy)
    return accuracy

In [8]:
pre_trained= None#'outputs/landsat_1000/landsat_30_last.tar' #None # last model save
stage = 'landsat/'#'mobility_1051/' #'mapillary/'#'landsat/'
save_dir = 'outputs/' + stage 
data_type = 'landsat'#'distance' # 'mobility' #'poi' #'sv'
save_path = save_dir + "training_log_" + data_type + ".txt"
node_list_path='../dataset/safegraph/graph_checkpoints/nyc_metro/node_list.csv'

In [9]:
if not os.path.exists(save_dir):
    createCleanDir(save_dir)

with open(save_path, "w") as file:
    file.write('')

In [10]:
# device = "cpu"
embedding_dim = 200
num_nodes = pd.read_csv(node_list_path).shape[0]
print(f'Number of nodes: {num_nodes}')
threshold = 0.6
return_best = True
if_early_stop = True
input_size = 299
learning_rate = [0.008]
weight_decay = [0.0005]
batch_size = 2
num_epochs = 60
lr_decay_rate = 0.7
lr_decay_epochs = 6
early_stop_epochs = 14
save_epochs = 3
margin=2

Number of nodes: 1089


In [11]:
import torchvision.transforms.functional as TF
import random
from torchvision.models import resnet18, ResNet18_Weights

class ResNetTransform:
    """Feed images through ResNet-18 model to get embeddings."""

    def __init__(self):
        super().__init__()
        weights = ResNet18_Weights.DEFAULT
        model = resnet18(weights=weights, progress=False)
        num_features = model.fc.in_features 
        model.fc = nn.Linear(num_features, 200)
#         model = FullyConvolutionalResnet18(pretrained=True)

        self.resnet18 = model.eval()
        self.resnet18 = self.resnet18.to(device)
        
        self.transforms =  torchvision.transforms.Compose([
#             torchvision.transforms.Resize((224, 224)),
#             torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
            

    def __call__(self, x):
        print('Begin')
        with torch.no_grad():
            x = torch.tensor(x)
            x = torch.nan_to_num(x, 0, 0, 0)
#             print(x.shape)
            x = x.to(device)
            print('1')
            print(type(x))
            print(x.shape)
            x = self.transforms(x)
            x = torch.unsqueeze(x, 0)
#             print(x.shape)
            with torch.no_grad():
#                 x = x.to(device)
                print('into model')
                print(x.shape)
                y_pred = self.resnet18(x)
                print('out of model')
                y_pred = torch.softmax(y_pred, dim=1)
            y_pred = torch.squeeze(y_pred)
            return y_pred

In [12]:
# landsat_model = ResNetTransform()
# # landsat_model.to(device)
# datasets1 = SatelliteImageryDataset(node_list_path=node_list_path, 
#                                     root_image_dir='../dataset/earth_engine/download_landsat_images', 
#                                     fn='least_cloudy_rectangle_highres.tif', is_train=True, transform=landsat_model)

In [13]:
sample1= datasets1.__getsampletest__(300)
sample1.shape

(3, 235, 378)

In [36]:
x = sample1
with torch.no_grad():
    x = torch.tensor(x)
    x = torch.nan_to_num(x, 0, 0, 0)
    x = x.to(device)
    print('1')
    print(type(x))
    print(x.shape)
    out = landsat_model.transforms(x).unsqueeze(0)

1
<class 'torch.Tensor'>
torch.Size([3, 235, 378])


In [37]:
out.shape

torch.Size([1, 3, 235, 378])

In [ ]:
landsat_model.resnet18(out)

In [ ]:
landsat_mode.unsqueeze(0)l.__call__(300)

In [28]:
# from torchvision.models import inception_v3, Inception_V3_Weights

# preprocess = transforms.Compose([
#     transforms.Resize(299),
#     transforms.CenterCrop(299),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

# class EncoderInception3(nn.Module):
#     def __init__(self, embedding_dim=200):
#         super(EncoderInception3, self).__init__()
#         self.model = torchvision.models.inception_v3(weights='DEFAULT').eval()
#         self.set_parameter_requires_grad(True)  # freeze all but last layer

#         # Handle the auxilary net
#         self.model.aux_logits = False

#         # Handle the primary net
#         num_ftrs = self.model.fc.in_features
#         self.model.fc = nn.Linear(num_ftrs, embedding_dim)

#     def forward(self, images):
#         x = self.model(images)  
#         return x

#     def set_parameter_requires_grad(self, feature_extracting):
#         if feature_extracting:
#             for param in self.model.parameters():
#                 param.requires_grad = False


In [14]:
from torchvision.models import inception_v3, Inception_V3_Weights
from torchvision import transforms

class Inceptionv3Transform:
    """Feed images through ResNet-18 model to get embeddings."""

    def __init__(self):
        super().__init__()
        weights = Inception_V3_Weights.DEFAULT
        model = inception_v3(weights=weights, progress=False)
        num_features = model.fc.in_features 
        model.fc = nn.Linear(num_features, 200)
        self.inception_v3 = model.eval()
        self.inception_v3 = self.inception_v3.to(device)
        
        self.transforms = transforms.Compose([
            transforms.Resize(299),
            transforms.CenterCrop(299),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
            

    def __call__(self, x):
        with torch.no_grad():
            x = torch.tensor(x).float()
            x = torch.nan_to_num(x, 0, 0, 0)
        
            if len(x.shape) != 3:
                print(x.shape)
                print(x)
            try:
                x = torch.reshape(x, (3, x.shape[0], x.shape[1]))
            except:
                print(x.shape)
                x = torch.reshape(x, (x.shape[2], x.shape[0], x.shape[1]))
            x = self.transforms(x)
            x = torch.unsqueeze(x, 0)
            with torch.no_grad():
#                 print(x.shape)
                x = x.to(device)
                y_pred = self.inception_v3(x)
            y_pred = torch.squeeze(y_pred)
            if(not np.isfinite(y_pred.cpu().detach().numpy()).all()):
                print('Invalid num in inception transform: {y_pred}')
            return y_pred

# torch.Size([3, 310, 306])
# torch.Size([1, 3, 224, 224])

In [15]:
# sv_model = Inceptionv3Transform()
# datasets1 = StreetViewDataset(node_list_path=node_list_path, 
#                                     root_image_dir='../dataset/mapillary/data/nyc_metro/', 
#                               is_train=True, transform=sv_model)


In [16]:
def train_embedding(model, data_type, dataloaders, criterion, optimizer, metrics, num_epochs,
                verbose=True, return_best=True, if_early_stop=True, early_stop_epochs=10, scheduler=None,
                save_dir=None, save_epochs=5):
    since = time.time()
    training_log = dict()
    training_log['loss_history'] = []
    training_log['metric_value_history'] = []
    training_log['current_epoch'] = -1
    current_epoch = training_log['current_epoch'] + 1

    best_model_wts = copy.deepcopy(model.state_dict())
    best_optimizer_wts = copy.deepcopy(optimizer.state_dict())
    best_log = copy.deepcopy(training_log)

    best_metric_value = 0
    nodecrease = 0  # to count the epochs that val loss doesn't decrease
    early_stop = False

    for epoch in tqdm(range(current_epoch, current_epoch + num_epochs)):
        if verbose:
            print('Epoch {}/{}'.format(epoch, num_epochs - 1))
            print('-' * 10)
        running_loss = 0.0
        stats = {'T':0,'F':0}

        # Iterate over data.
        for pos_index, pos_encoder_embedding, neg_encoder_embedding in dataloaders:
            # print(f"Feature batch shape: {pos_index.size()}")
            # print(f"Labels batch shape: {pos_encoder_embedding.size()}")

            pos_encoder_embedding = pos_encoder_embedding.to(device)
            neg_encoder_embedding = neg_encoder_embedding.to(device)
            pos_index = pos_index.to(device)

            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                # Get model outputs and calculate loss
                outputs = model(pos_index)

                loss = criterion(outputs, pos_encoder_embedding, neg_encoder_embedding) # anchor, pos, neg

                # evaluate based on whether dist(anchor, pos) < dist(anchor, neg)
                pos_dist = torch.nn.functional.pairwise_distance(outputs, pos_encoder_embedding, p=2.0) 
                neg_dist = torch.nn.functional.pairwise_distance(outputs, neg_encoder_embedding, p=2.0) 

                # backward + optimize only if in training phase
                loss.backward()
                optimizer.step()

            # statistics
            running_loss += loss.item()
            stats['T'] += torch.sum(pos_dist < neg_dist).cpu().item() # if pos distance < neg distance, this is good
            stats['F'] += torch.sum(pos_dist > neg_dist).cpu().item()

       
        epoch_loss = running_loss / len(dataloaders.dataset)
        epoch_metric_value = metrics(stats)
        if verbose:
            print('Loss: {:.5f} Metrics: {:.5f}'.format( epoch_loss, epoch_metric_value))

        training_log['current_epoch'] = epoch
        print()
        training_log['metric_value_history'].append(epoch_metric_value)
        training_log['loss_history'].append(epoch_loss)
        if epoch_metric_value > best_metric_value:
            best_metric_value = epoch_metric_value
            best_model_wts = copy.deepcopy(model.state_dict())
            best_optimizer_wts = copy.deepcopy(optimizer.state_dict())
            best_log = copy.deepcopy(training_log)
            nodecrease = 0
        else:
            nodecrease += 1
        if scheduler != None:
            scheduler.step()
        if nodecrease >= early_stop_epochs:
            early_stop = True
        if save_dir and epoch % save_epochs == 0:
            checkpoint = {
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'training_log': training_log
                }
            torch.save(checkpoint,os.path.join(save_dir, data_type + '_' + str(training_log['current_epoch']) + '.tar'))
        if if_early_stop and early_stop:
            print('Early stopped!')
            break

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best metric value: {:5f}'.format(best_metric_value))

    # plot full training curve
    best_epoch_loss = np.argmin(training_log["loss_history"])
    best_epoch_metric = np.argmax(training_log["metric_value_history"])
    plot_curves(training_log["loss_history"], training_log["metric_value_history"], best_epoch_loss=best_epoch_loss, best_epoch_metric=best_epoch_metric)

    # load best model weights
    if return_best:
        model.load_state_dict(best_model_wts)
        optimizer.load_state_dict(best_optimizer_wts)
        training_log = best_log

    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'training_log': training_log
    }

    if save_dir:
        torch.save(checkpoint,
                   os.path.join(save_dir, data_type + '_' + str(training_log['current_epoch']) + '_last.tar'))    
    return model, training_log, best_metric_value

In [17]:
torch.autograd.set_detect_anomaly(True)

In [ ]:
%timeit



best_metric=0
best_lr=-1
best_wr=-1

for i in learning_rate:
    for j in weight_decay:
        dataloaders_dict = DataLoader(datasets1, batch_size=batch_size,shuffle=True, num_workers=0)


        model = NodeEmbeddings(num_nodes, embedding_dim=200)
        for param in model.parameters():
            print(param.size())


        model = model.to(device)
        if pre_trained:
            checkpoint=torch.load(pre_trained)
            model.load_state_dict(checkpoint,strict=False)

        optimizer = optim.Adam(model.parameters(), lr=i, betas=(0.9, 0.999), eps=1e-08,
                               weight_decay=j, amsgrad=True)
        loss_fn = torch.nn.TripletMarginLoss(reduction="mean", margin=margin)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=lr_decay_epochs, gamma=lr_decay_rate)

        _, training_log,best_value = train_embedding(model, data_type=data_type, dataloaders=dataloaders_dict, criterion=loss_fn,
                               optimizer=optimizer, metrics=metrics, num_epochs=num_epochs,
                               save_dir=save_dir, verbose=True, return_best=return_best,
                               if_early_stop=if_early_stop, early_stop_epochs=early_stop_epochs, scheduler=scheduler,
                               save_epochs=save_epochs)
        
        
        print(training_log["metric_value_history"])

    if best_value>best_metric:
        best_metric=best_value
        best_lr=i
        best_wr=j

    with open(save_dir + 'metric_value_history.npy', 'wb') as f:
        np.save(f, training_log["metric_value_history"])
    with open(save_dir + 'loss_history.npy', 'wb') as f:
        np.save(f, training_log["loss_history"])

    best_epoch_loss = np.argmax(training_log["loss_history"])
    best_epoch_metric = np.argmax(training_log["metric_value_history"])
    with open(save_path, "a") as file:
        for k in range(len(training_log["metric_value_history"])):
            file.write("epoch:" + str(k)+"\n")
            file.write("metric_value_history:"+str(training_log["metric_value_history"][k])+"\n")
            file.write("loss_history:"+str(training_log["loss_history"][k])+"\n")

        file.write("\n\n---BEST---\nbest_lr:"+str(best_lr)+" best_wr:"+str(best_wr)+" best_metric_value:"+str(best_metric))

    print("best_lr:"+str(best_lr)+" best_wr:"+str(best_wr)+" best_metric_value:"+str(best_metric))

    model.print_history()

torch.Size([1089, 200])


  0%|          | 0/60 [00:00<?, ?it/s]

Epoch 0/59
----------
Begin
1
into model


In [25]:
pos = pos.cpu().detach()
np.isnan(pos)

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.uint8)

In [31]:
np.isnan(pos).sum()

tensor(0)

In [ ]:
# pos_image = io.imread('../dataset/mapillary/data/nyc_metro/36031961100/41.jpg')#.to_numpy(dtype='float64')

# t = transforms.Compose([
#             transforms.Resize(299),
#             transforms.CenterCrop(299),
#             transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#         ])
# x = np.array(pos_image)

# x = torch.tensor(x).float()


# x = x.to(device)
# x = torch.nan_to_num(x, 0, 0, 0)

# # x = x.float()

# x = torch.reshape(x, (3, x.shape[0], x.shape[1]))

In [ ]:

# best_metric=0
# best_lr=-1
# best_wr=-1

# for i in learning_rate:
#     for j in weight_decay:
#         model = NodeEmbeddings(num_nodes, embedding_dim=200)
#         for param in model.parameters():
#             print(param.size())
#         # media_dir=os.path.join(ckpt_save_dir,"lr%f_wr%f"%(i,j))
#         # if not os.path.exists(media_dir):
#         #     os.makedirs(media_dir)
#         model = model.to(device)
#         if pre_trained:
#             checkpoint=torch.load(pre_trained)
#             model.load_state_dict(checkpoint,strict=False)
            
#         optimizer = optim.Adam(model.parameters(), lr=i, betas=(0.9, 0.999), eps=1e-08,
#                                weight_decay=j, amsgrad=True)
#         loss_fn = torch.nn.TripletMarginLoss(reduction="mean", margin=margin)
#         scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=lr_decay_epochs, gamma=lr_decay_rate)

#         _, training_log,best_value = train_embedding(model, data_type=data_type, dataloaders=dataloaders_dict, criterion=loss_fn,
#                                optimizer=optimizer, metrics=metrics, num_epochs=num_epochs,
#                                save_dir=save_dir, verbose=True, return_best=return_best,
#                                if_early_stop=if_early_stop, early_stop_epochs=early_stop_epochs, scheduler=scheduler,
#                                save_epochs=save_epochs)

#         print(training_log["metric_value_history"])

#         if best_value>best_metric:
#             best_metric=best_value
#             best_lr=i
#             best_wr=j

#         with open(save_dir + 'metric_value_history.npy', 'wb') as f:
#             np.save(f, training_log["metric_value_history"])
#         with open(save_dir + 'loss_history.npy', 'wb') as f:
#             np.save(f, training_log["loss_history"])

#         best_epoch_loss = np.argmax(training_log["loss_history"])
#         best_epoch_metric = np.argmax(training_log["metric_value_history"])
#         with open(save_path, "a") as file:
#             for k in range(len(training_log["metric_value_history"])):
#                 file.write("epoch:" + str(k)+"\n")
#                 file.write("metric_value_history:"+str(training_log["metric_value_history"][k])+"\n")
#                 file.write("loss_history:"+str(training_log["loss_history"][k])+"\n")

#             file.write("\n\n---BEST---\nbest_lr:"+str(best_lr)+" best_wr:"+str(best_wr)+" best_metric_value:"+str(best_metric))

#         print("best_lr:"+str(best_lr)+" best_wr:"+str(best_wr)+" best_metric_value:"+str(best_metric))

# model.print_history()


In [ ]:
# print(training_log["metric_value_history"])

# if best_value>best_metric:
#     best_metric=best_value
#     best_lr=i
#     best_wr=j

# with open(save_dir + 'metric_value_history.npy', 'wb') as f:
#     np.save(f, training_log["metric_value_history"])
# with open(save_dir + 'loss_history.npy', 'wb') as f:
#     np.save(f, training_log["loss_history"])

# best_epoch_loss = np.argmax(training_log["loss_history"])
# best_epoch_metric = np.argmax(training_log["metric_value_history"])
# with open(save_path, "a") as file:
#     for k in range(len(training_log["metric_value_history"])):
#         file.write("epoch:" + str(k)+"\n")
#         file.write("metric_value_history:"+str(training_log["metric_value_history"][k])+"\n")
#         file.write("loss_history:"+str(training_log["loss_history"][k])+"\n")

#     file.write("\n\n---BEST---\nbest_lr:"+str(best_lr)+" best_wr:"+str(best_wr)+" best_metric_value:"+str(best_metric))

# print("best_lr:"+str(best_lr)+" best_wr:"+str(best_wr)+" best_metric_value:"+str(best_metric))

# model.print_history()